# Trying Out and Ensembling Different Classification Models 

**This is the second complete notebook that I am creating. [Here's a link to the first](https://github.com/elijahrona/Elijah-Rona-ML-Journey/blob/master/do-you-have-malaria-or-covid-19.ipynb) where I worked with Covid-19, Malaria, and Negative patients.**

**The purpose of this notebook is to improve my Recipes(), Workflow(), and Ensembling (Stacks()) skills.**

# Importing Libraries


In [ ]:
library(tidyverse)
library(tidymodels)


# Dataset



In [ ]:
mine <- read.csv("C:/Users/Octopus/Desktop/in-vehicle-coupon-recommendation.csv", stringsAsFactors=TRUE)
head(mine)


Let's have a glimpse of what our dataset looks like.



In [ ]:
skimmed <- skimr::skim(mine)
skimmed <- skimmed[, c(1:5, 9:11, 13, 15)]
skimmed


The response variable, Y appears to be numeric. It should be a factor, so we should convert it with factor(). Also, it seems that the temperature column has only three numbers, so we should also treat it as a factor.



In [ ]:
mine <- mine %>%
mutate(temperature = factor(temperature),
      Y = factor(Y))


In [ ]:
is.factor(mine$Y)



# Splitting the Dataset



In [ ]:
ames_split  <- initial_split(mine, 
                             strata = Y,
                             breaks = 4)
ames_train  <- training(ames_split)
ames_test   <- testing(ames_split)


# Creating the Recipe



In [ ]:
train_rec <- recipe(Y ~ ., data = ames_train) %>%
step_dummy(all_nominal_predictors())


Let's check if the recipe works. The data below is what it looks like after processing it with recipe. This is the way our models will read it



In [ ]:
train_rec %>% 
prep(training = ames_train, retain = TRUE) %>%
juice() %>%
head()


# Specifying Our Models

We shall be working with four models; Logistic Regression, MARS, Random Forest (named treebag), and XGBOOST


In [ ]:
log_spec <- logistic_reg(penalty = 10) %>%
  set_engine(engine = "glm") %>%
  set_mode("classification")

mars_spec <- mars() %>%
  set_mode("classification") %>% 
  set_engine("earth")

treebag_spec <- rand_forest() %>%
  set_engine("ranger") %>% 
  set_mode("classification")

xgboost_spec <- boost_tree() %>% 
  set_mode("classification") %>% 
  set_engine("xgboost")


# Adding Our Models and Recipe Into Various Workflows

A workflow is created for each model, but with the same recipe


In [ ]:
wf_log <- workflow() %>% 
add_recipe(train_rec) %>% 
add_model(log_spec)

wf_mars <- workflow() %>% 
add_recipe(train_rec) %>% 
add_model(mars_spec)

wf_treebag <- workflow() %>% 
add_recipe(train_rec) %>% 
add_model(treebag_spec)

wf_xgboost <- workflow() %>% 
add_recipe(train_rec) %>% 
add_model(xgboost_spec)


# Building Our Logistic Model

First of all, let us start with fitting the train data


In [ ]:
train_fit_log <- 
  wf_log %>% 
  fit(data = ames_train)


Now we move to predicting the test data. Note that the data was also processed with the recipe



In [ ]:
pred_log <- augment(train_fit_log, ames_test)



What is the accuracy of our model prediction?



In [ ]:
LOG_Accuracy <- pred_log %>% 
  accuracy(truth = Y, .pred_class)

LOG_Accuracy[[1,3]]


With an accuracy of 56.8%, there s room for improvement. Let's plot a confusion matrix for the model for better visualization.



In [ ]:
p1 <- conf_mat(pred_log, truth = Y, estimate = .pred_class) %>% 
  autoplot(type = "heatmap") +
  labs(title = "Logistic",
       subtitle = LOG_Accuracy[[1,3]])

p1


The Logistic model is very poor. It could not detect any 0


# Building Our MARS Model


In [ ]:
train_fit_mars <- 
  wf_mars %>% 
  fit(data = ames_train)

pred_mars <- augment(train_fit_mars, ames_test)

MARS_Accuracy <- pred_mars %>% 
  accuracy(truth = Y, .pred_class)

MARS_Accuracy[[1,3]]

p2 <- conf_mat(pred_mars, truth = Y, estimate = .pred_class) %>% 
  autoplot(type = "heatmap") +
  labs(title = "MARS",
       subtitle = MARS_Accuracy[[1,3]])

p2


# Building Our Random Forest Model



In [ ]:
train_fit_treebag <- 
  wf_treebag %>% 
  fit(data = ames_train)

pred_treebag <- augment(train_fit_treebag, ames_test)

TREEBAG_Accuracy <- pred_treebag %>% 
  accuracy(truth = Y, .pred_class)

TREEBAG_Accuracy[[1,3]]

p3 <- conf_mat(pred_treebag, truth = Y, estimate = .pred_class) %>% 
  autoplot(type = "heatmap") +
  labs(title = "Random Frest",
       subtitle = TREEBAG_Accuracy[[1,3]])

p3


# Building Our XGBOOST Model



In [ ]:
train_fit_xgboost <- 
  wf_xgboost %>% 
  fit(data = ames_train)

pred_xgboost <- augment(train_fit_xgboost, ames_test)

XGBOOST_Accuracy <- pred_xgboost %>% 
  accuracy(truth = Y, .pred_class)

XGBOOST_Accuracy[[1,3]]

p4 <- conf_mat(pred_xgboost, truth = Y, estimate = .pred_class) %>% 
  autoplot(type = "heatmap") +
  labs(title = "XGBOOST",
       subtitle = XGBOOST_Accuracy[[1,3]])

p4


# Confusion Matrix for Every Model

We can see that the Random Forest model is best while the MARS is the worst.


In [ ]:
ggpubr::ggarrange(p1,p2,p3,p4,
                   ncol = 2,
                   nrow = 2)


# Ensembling Our Models

Let us start by creating our cross validation folds


In [ ]:
ctrl_grid <- stacks::control_stack_grid()
ctrl_res <- stacks::control_stack_resamples()

folds <- rsample::vfold_cv(ames_train, v = 5)

metric <- metric_set(accuracy, roc_auc)


# Preparing the MARS Model for Ensembling



In [ ]:
mars_res <- 
  fit_resamples(
    wf_mars, #workflow
    resamples = folds, #cvfold
    metrics = metric,
    control = ctrl_res
  )


# Preparing the Random Forest Model for Ensembling



In [ ]:
treebag_res <- 
  fit_resamples(
    wf_treebag, #workflow
    resamples = folds, #cvfold
    metrics = metric,
    control = ctrl_res
  )


# Preparing the XGBOOST Model for Ensembling



In [ ]:
xgboost_res <- 
  fit_resamples(
    wf_xgboost, #workflow
    resamples = folds, #cvfold
    metrics = metric,
    control = ctrl_res
  )


# Preparing the Logistic Model for Ensembling



In [ ]:
log_res <- 
  fit_resamples(
    wf_log, #workflow
    resamples = folds, #cvfold
    metrics = metric,
    control = ctrl_res
  )


# Adding Every Model to Our Stack



In [ ]:
library(stacks)
model_data_st <-  stacks() %>%
  add_candidates(log_res) %>%
  add_candidates(treebag_res) %>%
  add_candidates(xgboost_res) %>%
  add_candidates(mars_res)

head(model_data_st)


There are several class probabilities. To know the combined model prediction, we will use the blend_predictions() function.



In [ ]:
fitted_model_st <-
  model_data_st %>%
  blend_predictions()


Let us expore our ensambled model to know how the members are performing.



In [ ]:
theme_set(theme_bw())
autoplot(fitted_model_st)


In [ ]:
autoplot(fitted_model_st, type = "members")



In [ ]:
autoplot(fitted_model_st, type = "weights")



In [ ]:
fitted_model_st



The final model retained just two of our models; Random Forest and XGBOOST. Let us combine these models to predict our test data



In [ ]:
fitted_model_st <-
  fitted_model_st %>%
  fit_members()


In [ ]:
test_predict_data <- 
  ames_test %>%
  bind_cols(predict(fitted_model_st, .))


Prediting the test data



In [ ]:
member_preds <- 
  test_predict_data %>%
  select(Y) %>%
  bind_cols(predict(fitted_model_st, ames_test, members = TRUE))

head(member_preds)


Let us compare the accuracy of the combined model with that of the other member models



In [ ]:
map_dfr(member_preds, accuracy, truth = Y, data = member_preds) %>%
  mutate(member = colnames(member_preds))


After every model learned from each other, the model with the highest accuracy is Random Forest (74.55%) while the combined model came second (74.14%). XGBOOST came last (72.06%).



In [ ]:
p1 <- conf_mat(member_preds, truth = Y, estimate = .pred_class) %>% 
  autoplot(type = "heatmap") +
  labs(title = "Ensembled")

p2 <- conf_mat(member_preds, truth = Y, estimate = .pred_class_treebag_res_1_1) %>% 
  autoplot(type = "heatmap") +
  labs(title = "Random Forest")

p3 <- conf_mat(member_preds, truth = Y, estimate = .pred_class_xgboost_res_1_1) %>% 
  autoplot(type = "heatmap") +
  labs(title = "XGBOOST")

ggpubr::ggarrange(p1,p2,p3,
          ncol = 2,
          nrow = 2)


# Thanks for Reading

